In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [13]:
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.width = 2000
pd.options.display.max_columns = 10

In [2]:
def convert_price_to_float(data, data_dictionary):
    """
    Function to convert string price columns into float price columns
    :param data: type of data set
    :param data_dictionary: dictionary with airbnb dataframes
    :return: dictionary containing updated dataframes with float price columns
    """
    string_columns = data_dictionary[data].select_dtypes(include=['object']).columns
    price_columns = [col for col in string_columns if 'price' in col]
    if len(price_columns) > 0:
        for price in price_columns:
            data_dictionary[data][price + '_float'] = data_dictionary[data][price]\
                .str\
                .replace('$', '')\
                .str\
                .replace(',', '')\
                .astype(float)
    return data_dictionary

In [3]:
def split_date(data, data_dictionary):
    """
    Function to split date column into separate year, month, day columns
    :param data: type of data set
    :param data_dictionary: dictionary with airbnb dataframes
    :return: dictionary containing updated dataframes with new date columns
    """
    if 'date' in data_dictionary[data].select_dtypes(include=['object']).columns:
        data_dictionary[data][['year','month','day']] = data_dictionary[data]['date']\
            .str.split('-', 3, expand=True)
    return data_dictionary

In [4]:
def load_csv(cities, data_sets, source_directory):
    """
    Function to load three types of csv data sets for Boston and Seattle
    :param city_type: list of city names
    :param data_type: list of data sets
    :param source_directory: location of the csv data sets
    :return: 
    """
    city_dictionary = {}
    for city in cities:
        data_dictionary = {}
        for data in data_sets:
            file_path = source_directory + '/' + data + '-' + city + '.csv'
            # 1. read csv into dataframe
            data_dictionary[data] = pd.read_csv(file_path)
            # 2. convert string price columns into float columns
            data_dictionary = convert_price_to_float(data, data_dictionary)
            # 3. split date column into separate year, month, day column
            data_dictionary = split_date(data, data_dictionary)
        city_dictionary[city] = data_dictionary
    return city_dictionary

In [5]:
# 1. load files
data_type = ['calendar'
             , 'listings'
             , 'reviews']

city_type = ['seattle'
             , 'boston']
city_dictionary = load_csv(city_type, data_type, 'data')

In [6]:
print(city_dictionary.keys())

dict_keys(['seattle', 'boston'])


In [7]:
def concatenate_dataframes(dictionary, data):
    """
    Function to get combine datasets including data for each city
    :param dictionary: complete data dictionary
    :param data: particular data of interest
    :return: dataframe including both cities
    """
    seattle_df = dictionary['seattle'][data]
    boston_df = dictionary['boston'][data]
    seattle_df['boston'] = 0
    boston_df['boston'] = 1
    intersection_columns = set(seattle_df.columns).intersection(set(boston_df.columns))
    df = pd.concat([seattle_df[intersection_columns], boston_df[intersection_columns]])
    return df

In [8]:
reviews_df = concatenate_dataframes(city_dictionary, 'reviews')
listings_df = concatenate_dataframes(city_dictionary, 'listings')

In [9]:
len(listings_df.columns)

96

In [10]:
reviews_df['boston'].value_counts()

0    84849
1    68275
Name: boston, dtype: int64

In [11]:
def check_missing_values(df, missing_values_maximum):
    """
    Function to check what columns have the most amount of missing data
    :param df: dataframe with data to compare
    :param missing_values_maximum: value spevifying the cutoff for missing data
    :return: list of columns that have more missing data than specified amount
    """
    seattle_data = df[df['boston'] == 0].isnull().mean().reset_index()
    boston_data = df[df['boston'] == 1].isnull().mean().reset_index()
    boston_data.columns = ['index', 'boston']
    seattle_data.columns = ['index', 'seattle']
    merged_data = seattle_data.merge(boston_data, how='inner', on='index')
    merged_data = merged_data.sort_values(by=['boston', 'seattle'], ascending=False)
    print(merged_data.head(30))
    missing_data_list = merged_data[(merged_data['boston'] > missing_values_maximum) 
                               | (merged_data['seattle'] > missing_values_maximum)]['index'].tolist()
    return missing_data_list

In [12]:
def drop_specified_columns(key, df):
    """
    Function to drop columns that include a specified substring
    :param key: substring to specify columns to delete 
    :param df: dataframe to remove columns from
    :return: updated dataframe
    """
    drop_columns = [col for col in df.columns if key in col.lower()]
    if len(drop_columns) > 0:
        df = df.drop(columns=drop_columns)
    return df

In [ ]:
def compare_float_columns(df):
    columns_to_check = df\
            .select_dtypes(include=['bool', 'int64', 'float64'])\
            .columns
    for column in columns_to_check:
        if 'id' not in column:
            print('\n Describe ' + column)
            seattle_data = df[df['boston'] == 0][column].describe().reset_index()
            seattle_data.columns = ['index', column + '_seattle']
            boston_data = df[df['boston'] == 1][column].describe().reset_index()
            boston_data.columns = ['index', column + '-boston']
            merged_data = seattle_data.merge(boston_data, how='inner', on='index')
            print(merged_data.head(10))